In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=50
Nul=100
Nup=10**(40)
scal=1
umax=8
vmax=200
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 800000000,80000X10000
Total number of points is 800000000
Runtime: about 170.94 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)




Using Standard Coordinates Along V


In [6]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)
drunp=np.array([0.0],dtype=datatype)
dsigunp=np.array([0.0],dtype=datatype)
dphinpuf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigunpvalue=100*np.exp(np.nanmax(signpf))
drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigunp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=M0
    
    
    

    
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
    
            
            
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    print(TempTol)
    print(exterm1)
    print(exterm2)
    print(exterm3)
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.000200000000000000009584
1
-1.55995518726
0.150416619442
0.452916200681
---
0.000200000000000000009584
0.010000000000000000208167
1
-1.5635727586
0.150414204375
0.454487511292
---
0.010200000000000000732747
0.010000000000000000208167
1
-1.57071953511
0.150409455574
0.457592007342
---
0.020200000000000002675637
0.010000000000000000208167
1
-1.57794824125
0.150404680381
0.460731602161
---
0.030200000000000004618528
0.010000000000000000208167
1
-1.58526026741
0.150399878478
0.463906872249
---
0.040200000000000006561418
0.010000000000000000208167
1
-1.59265703541
0.150395049537
0.467118406774
---
0.050200000000000008504308
0.010000000000000000208167
1
-1.60013999935
0.150390193224
0.470366807926
---
0.060200000000000010447199
0.010000000000000000208167
1
-1.60771064631
0.150385309202
0.473652691277
---
0.070200000000000012390089
0.010000000000000000208167
1
-1.61537049755
0.150380397126
0.476976686158
---
0.080200000000000007394085
0.010000000000000000208167
1


0.841463555767
---
0.750739982447848563928972
0.008109523382304879152160
1
-2.48325378303
0.149952924187
0.84865530247
---
0.758849505830153425733897
0.008053949272793948371829
1
-2.50041698514
0.14994623623
0.855904953822
---
0.766903455102947329002916
0.007998665870090385446312
1
-2.51772696239
0.149939539522
0.863212936943
---
0.774902120973037678020034
0.007943673122128010882781
1
-2.53518489554
0.149932834047
0.870579682292
---
0.782845794095165636861111
0.007888970952460206548240
1
-2.55279197426
0.149926119787
0.87800562369
---
0.790734765047625853817692
0.007834559259693453861506
1
-2.57054939666
0.149919396726
0.88549119836
---
0.798569324307319305944475
0.007780437919602016354215
1
-2.58845836992
0.149912664847
0.893036846957
---
0.806349762226921340513286
0.007726606783561634425594
1
-2.60652011009
0.149905924134
0.900643013609
---
0.814076369010482969734710
0.007673065679637458037787
1
-2.62473584218
0.149899174569
0.90831014594
---
0.821749434690120406088454
0.007619814412

1.64102422899
---
1.247184316703023876371503
0.004545913561225826612922
1
-4.43275970262
0.149388798933
1.65453080753
---
1.251730230264249676963573
0.004511861987054981842915
1
-4.46623709352
0.149381354644
1.668143142
---
1.256242092251304676153723
0.004478042607500334545967
1
-4.49999003024
0.149373900125
1.6818620531
---
1.260720134858805074884458
0.004444454291140451616948
1
-4.53402066968
0.149366435355
1.69568836842
---
1.265164589149945451040935
0.004411095902966881247831
1
-4.56833118523
0.149358960314
1.70962292245
---
1.269575685052912294992211
0.004377966305217904398328
1
-4.60292376724
0.149351474982
1.72366655666
---
1.273953651358130212400965
0.004345064357215037108284
1
-4.63780062339
0.149343979337
1.73782011954
---
1.278298715715345146293203
0.004312388915369153204182
1
-4.67296397786
0.149336473358
1.75208446666
---
1.282611104630714260466107
0.004279938834275335071688
1
-4.70841607292
0.149328957024
1.76646046074
---
1.286891043464989614619753
0.00424771296551528908

3.16073457682
---
1.522975822918365196301238
0.002442269999187951946135
1
-8.25306702194
0.148750800845
3.18615744432
---
1.525418092917553147813692
0.002423341522226842795901
1
-8.31754583872
0.148742455609
3.21178037244
---
1.527841434439779932930037
0.002404555428704267024814
1
-8.38254326333
0.148734098259
3.2376049624
---
1.530245989868484146612104
0.002385910739941755204230
1
-8.44806337632
0.14872572877
3.26363282854
---
1.532631900608425912224675
0.002367406482301609958740
1
-8.51411029218
0.148717347114
3.28986559847
---
1.534999307090727516111883
0.002349041686524253307738
1
-8.58068815772
0.148708953266
3.31630491311
---
1.537348348777251683117129
0.002330815388275443884869
1
-8.64780115275
0.148700547199
3.34295242692
---
1.539679164165527192054128
0.002312726628044661607297
1
-8.71545349134
0.148692128887
3.36980980784
---
1.541991890793571906570492
0.002294774450906286993729
1
-8.78364942063
0.148683698303
3.39687873755
---
1.544286665244478218284030
0.0022769579069289174

5.98191643239
---
1.668810574761170562041457
0.001305205048682579489078
1
-15.4443270354
0.148042621204
6.0296738223
---
1.670115779809853151505195
0.001294973873198269160786
1
-15.5663578447
0.14803321726
6.07781114053
---
1.671410753683051364504308
0.001284822063039796390463
1
-15.6893629416
0.148023798837
6.12633145818
---
1.672695575746091067870225
0.001274749017815390373168
1
-15.8133500781
0.148014365901
6.17523787149
---
1.673970324763906392107060
0.001264754141355770499996
1
-15.9383270691
0.148004918419
6.22453350198
---
1.675235078905262131598874
0.001254836841611249066697
1
-16.0643017927
0.147995456357
6.27422149665
---
1.676489915746873338164846
0.001244996530699382609672
1
-16.1912821904
0.147985979681
6.32430502826
---
1.677734912277572654204505
0.001235232624863813639948
1
-16.3192762685
0.147976488358
6.37478729544
---
1.678970144902436567591053
0.001225544544437754930846
1
-16.448292097
0.147966982353
6.42567152283
---
1.680195689446874363071061
0.00121593171388389639

...

1
-7.39459023469e-07
1.72278857846e-11
-3.1499053242
---
3.878637792991718846735694
0.006349397185487584173436
1
-7.41333061328e-07
1.72278857846e-11
-3.14885741467
---
3.884987190177206617391903
0.006351510203922225775153
1
-7.43179310727e-07
1.72278857846e-11
-3.14790539038
---
3.891338700381128834493438
0.006353431097749946687703
1
-7.44996770956e-07
1.72251102271e-11
-3.14704966712
---
3.897692131478878607708793
0.006355158677334789459057
1
-7.46785414618e-07
1.72278857846e-11
-3.14629066713
---
3.904047290156213545486708
0.006356691773248849750000
1
-7.48547059393e-07
1.72334368997e-11
-3.14562881858
---
3.910403981929462258193553
0.006358029237866257357692
1
-7.50277394535e-07
1.72362124573e-11
-3.14506455588
---
3.916762011167328694227763
0.006359169945369675357172
1
-7.51978408649e-07
1.72362124573e-11
-3.14459831999
---
3.923121181112698252491100
0.006360112791778335834814
1
-7.53652649296e-07
1.72362124573e-11
-3.14423055798
---
3.929481293904476757461453
0.006360856696484852

-3.3657729934
---
4.340811060026047663029658
0.005942171393974416890305
1
-8.77861291561e-07
1.72778458207e-11
-3.37336180015
---
4.346753231420022345332654
0.005928803723077143150721
1
-8.83476832169e-07
1.72750702632e-11
-3.38109399656
---
4.352682035143099703589087
0.005915245189974558184809
1
-8.89428799584e-07
1.72722947056e-11
-3.38897036354
---
4.358597280333074586167186
0.005901497462220267609112
1
-8.95734134534e-07
1.7269519148e-11
-3.39699167777
---
4.364498777795295048065327
0.005887562260132954070069
1
-9.0240973341e-07
1.7269519148e-11
-3.4051587108
---
4.370386340055428142647997
0.005873441357237840082517
1
-9.09474266386e-07
1.72667435905e-11
-3.4134722283
---
4.376259781412666249877930
0.005859136580697340071500
1
-9.16951774739e-07
1.72611924754e-11
-3.42193298913
---
4.382118917993363282903374
0.005844649811539585904097
1
-9.24854721705e-07
1.72584169178e-11
-3.43054174501
---
4.387963567804902709212911
0.005829982984201959560211
1
-9.33207933034e-07
1.72611924754e-1

4.739419392744059322808425
0.004584223495361780358059
1
-3.61068568991e-06
1.72917236085e-11
-4.38176077942
---
4.744003616239421461386883
0.004564375146614041076076
1
-3.68827720669e-06
1.7288948051e-11
-4.40085507484
---
4.748567991386035913592423
0.004544571375308203639576
1
-3.76656931104e-06
1.7288948051e-11
-4.42007001163
---
4.753112562761343973249950
0.004524815205951348597024
1
-3.84551833657e-06
1.72944991661e-11
-4.4394036544
---
4.757637377967295577718687
0.004505109595108919805229
1
-3.92507397611e-06
1.72944991661e-11
-4.45885404369
---
4.762142487562404546963535
0.004485457430099955636327
1
-4.0051923861e-06
1.72917236085e-11
-4.47841919747
---
4.766627944992504417598411
0.004465861527943405795416
1
-4.0858192711e-06
1.7288948051e-11
-4.49809711471
---
4.771093806520448232788567
0.004446324632385883005026
1
-4.16691776833e-06
1.72861724934e-11
-4.51788577694
---
4.775540131152833822625325
0.004426849413077930429916
1
-4.24842869079e-06
1.72861724934e-11
-4.5377831499
---

1.72722947056e-11
-5.99906473903
---
5.033552351082747122745786
0.003333853003763629068573
1
-8.67269491441e-06
1.72722947056e-11
-6.02242050874
---
5.036886204086510332444959
0.003320923866241015480483
1
-8.70840103508e-06
1.72722947056e-11
-6.04579573987
---
5.040207127952751164912115
0.003308083974474165279978
1
-8.74320035165e-06
1.72667435905e-11
-6.06919028399
---
5.043515211927225294630261
0.003295332501397234270524
1
-8.77709354958e-06
1.72639680329e-11
-6.09260402046
---
5.046810544428622868906587
0.003282668614736910985691
1
-8.81010817076e-06
1.72611924754e-11
-6.11603685605
---
5.050093213043359341440919
0.003270091477654394045843
1
-8.84224244515e-06
1.72584169178e-11
-6.1394887237
---
5.053363304521013787962147
0.003257600249806999557350
1
-8.87352311438e-06
1.72556413602e-11
-6.16295958194
---
5.056620904770820601470405
0.003245194088015955925092
1
-8.90395369837e-06
1.72528658027e-11
-6.18644941557
---
5.059866098858836203078226
0.003232872146283021635993
1
-8.933549364

---
5.250183750667184234828255
0.002566490802149549738803
1
-9.66987023037e-06
1.71834768636e-11
-7.81827849719
---
5.252750241469334113730838
0.002558107901525528277142
1
-9.67057522843e-06
1.71862524212e-11
-7.84386372835
---
5.255308349370859666294109
0.002549763827194947551219
1
-9.67114018438e-06
1.71862524212e-11
-7.86949803053
---
5.257858113198054184067587
0.002541458161933463511667
1
-9.6715743016e-06
1.71862524212e-11
-7.89518199937
---
5.260399571359987369589817
0.002533190495367288689460
1
-9.6718788044e-06
1.71862524212e-11
-7.92091623081
---
5.262932761855354968361098
0.002524960423418149205177
1
-9.67207239905e-06
1.71834768636e-11
-7.94670132086
---
5.265457722278773289303899
0.002516767548252738396358
1
-9.6721429014e-06
1.71779257485e-11
-7.97253786442
---
5.267974489827025763588608
0.002508611478567181543586
1
-9.67210242748e-06
1.7175150191e-11
-7.99842645587
---
5.270483101305592832375169
0.002500491829278471064835
1
-9.67195315863e-06
1.7175150191e-11
-8.024367689

In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

5157


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

915.5517953386585 seconds
15.259196588977641 minutes
0.25431994314962736 hours
51570000 points
56326.687646245606 points per second
Took 155.68 less minutes


conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            